## 一、Model

## 1.模型
   Word2Vec模型中，主要有Skip-Gram和CBOW两种模型，从直观上理解，Skip-Gram是给定input word来预测上下文。而CBOW是给定上下文，来预测input word。本篇文章仅讲解Skip-Gram模型。

<img src="images/CBOW_Skipgram.jpeg" style="width:500;height:300px;">
<caption><center> **Figure 1**: CBOW Skip-gram model </center></caption>

   Skip-Gram模型的基础形式非常简单，为了更清楚地解释模型，我们先从最一般的基础模型来看Word2Vec（下文中所有的Word2Vec都是指Skip-Gram模型）。

   Word2Vec模型实际上分为了两个部分，第一部分为建立模型，第二部分是通过模型获取嵌入词向量。Word2Vec的整个建模过程实际上与自编码器（auto-encoder）的思想很相似，即先基于训练数据构建一个神经网络，当这个模型训练好以后，我们并不会用这个训练好的模型处理新的任务，我们真正需要的是这个模型通过训练数据所学得的参数，例如隐层的权重矩阵——后面我们将会看到这些权重在Word2Vec中实际上就是我们试图去学习的“word vectors”。基于训练数据建模的过程，我们给它一个名字叫“Fake Task”，意味着建模并不是我们最终的目的。

   上面提到的这种方法实际上会在无监督特征学习（unsupervised feature learning）中见到，最常见的就是自编码器（auto-encoder）：通过在隐层将输入进行编码压缩，继而在输出层将数据解码恢复初始状态，训练完成后，我们会将输出层“砍掉”，仅保留隐层。

## 2.样本
我们选定句子“The quick brown fox jumps over lazy dog”，设定我们的窗口大小为2（window_size=2），也就是说我们仅选输入词前后各两个词和输入词进行组合。下图中，蓝色代表input word，方框内代表位于窗口内的单词。

<img src="images/source_text_To_training_samples.jpeg" style="width:500;height:300px;">
<caption><center> **Figure 2**: Generate training samples </center></caption>

## 3. Nueral network structure
<img src="images/structure.jpeg" style="width:500;height:300px;">
<caption><center> **Figure 3**: Nueral network structure(We assume the size of the vocabulary is equal to 10000)</center></caption>

模型的输入如果为一个10000维的向量，那么输出也是一个10000维度（词汇表的大小）的向量，它包含了10000个概率，每一个概率代表着当前词是输入样本中output word的概率大小。

## 3.1 Hidden layer
如果我们现在想用300个特征来表示一个单词（即每个词可以被表示为300维的向量）。那么隐层的权重矩阵应该为10000行，300列（隐层有300个结点）。

看下面的图片，左右两张图分别从不同角度代表了输入层-隐层的权重矩阵。左图中每一列代表一个10000维的词向量和隐层单个神经元连接的权重向量。从右边的图来看，每一行实际上代表了每个单词的词向量。

<img src="images/weights vector.jpeg" style="width:500;height:400px;">
<caption><center> **Figure 3**: Weights matrix</center></caption>

所以我们最终的目标就是学习这个隐层的权重矩阵。

我们现在回来接着通过模型的定义来训练我们的这个模型。

上面我们提到，input word和output word都会被我们进行one-hot编码。仔细想一下，我们的输入被one-hot编码以后大多数维度上都是0（实际上仅有一个位置为1），所以这个向量相当稀疏，那么会造成什么结果呢。如果我们将一个1 x 10000的向量和10000 x 300的矩阵相乘，它会消耗相当大的计算资源，为了高效计算，它仅仅会选择矩阵中对应的向量中维度值为1的索引行，看图就明白。
<img src="images/one_hot_To_word_vec.jpeg" style="width:300;height:100px;">
<caption><center> **Figure 4**: Transform</center></caption>

为了有效地进行计算，这种稀疏状态下不会进行矩阵乘法计算，可以看到矩阵的计算的结果实际上是矩阵对应的向量中值为1的索引，上面的例子中，左边向量中取值为1的对应维度为3（下标从0开始），那么计算结果就是矩阵的第3行（下标从0开始）—— [10, 12, 19]，这样模型中的隐层权重矩阵便成了一个”查找表“（lookup table），进行矩阵计算时，直接去查输入向量中取值为1的维度下对应的那些权重值。隐层的输出就是每个输入单词的“嵌入词向量”。

## 3.2 Output layer
经过神经网络隐层的计算，input word会从一个1 x 10000的向量变成1 x 300的向量，再被输入到输出层。输出层是一个softmax回归分类器，它的每个结点将会输出一个0-1之间的值（概率），这些所有输出层神经元结点的概率之和为1。

## 二、Implement

## 1 - Import packages

In [1]:
import numpy as np
import tensorflow as tf
import time
import random
from collections import Counter

## 2 - Loading data

In [2]:
with open('data/text8.txt') as f:
    text = f.read()

## 3 - Preprocess

In [3]:
def preprocess(text, freq=5):
    """Preprocess text.
    
    Arguments:
        text: text to be preprocessed
        freq: a threshold of frequency one word appears
        
    Returns:
        list
    """
    text = text.lower()
    text = text.replace('.', ' <PERIOD> ')
    text = text.replace(',', ' <COMMA> ')
    text = text.replace('"', ' <QUOTATION_MARK> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT_PAREN> ')
    text = text.replace('--', ' <HYPHENS> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace(':', ' <COLON> ')
    
    words = text.split()
    
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > freq]
    
    return trimmed_words

In [4]:
words = preprocess(text)

In [5]:
print(words[:20])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english']


In [6]:
# Construct the vocabulary
vocab = set(words)
vocab_to_index = {w: i for i, w in enumerate(vocab)}
index_to_vocab = {i: w for i, w in enumerate(vocab)}

In [7]:
print(vocab_to_index['a'])
print(index_to_vocab[1])
print(len(index_to_vocab))

46253
eir
63641


In [8]:
print('total words: ', len(words))
print('unique words: ', len(set(words)))

total words:  16680599
unique words:  63641


In [9]:
# words ---> int
words_int = [vocab_to_index[word] for word in words]

In [10]:
print(words[:10])
print(words_int[:10])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']
[17280, 20171, 54572, 46253, 35289, 11579, 40865, 50835, 29246, 43458]


## 4 - Sample
对停用词进行采样，例如“the”， “of”以及“for”这类单词进行剔除。剔除这些单词以后能够加快我们的训练过程，同时减少训练过程中的噪音。

我们采用以下公式: $$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

其中$ t $是一个阈值参数，一般为1e-3至1e-5。

$f(w_i)$ 是单词 $w_i$ 在整个数据集中的出现频次。

$P(w_i)$ 是单词被删除的概率。

In [11]:
t = 1e-5
threshold = 0.8
c = Counter(words_int)
total_counts = len(words_int)
freq_of_words = {w: word_count / total_counts for w, word_count in c.items()}
# The probability of word to be deleted
prop_of_words = {w: 1 - np.sqrt(t / freq_of_words[w]) for w in words_int}
sampled_words = [w for w in words_int if prop_of_words[w] < threshold]

In [12]:
print(len(sampled_words))

6925252


## 5 - Get batch
构造我们的训练样本。

对于一个给定词，离它越近的词可能与它越相关，离它越远的词越不相关，这里我们设置窗口大小为5，对于每个训练单词，我们还会在[1:5]之间随机生成一个整数R，用R作为我们最终选择output word的窗口大小。这里之所以多加了一步随机数的窗口重新选择步骤，是为了能够让模型更聚焦于当前input word的邻近词。

In [13]:
def get_target_words(words, idx, window_size=5):
    """Get target words corresponding to a given input word
    
    Arguments:
        words: words list
        idx: index of the given input word
        window_size: default to 5
    
    Return:
        return the target words
    """
    random_window_size = np.random.randint(1, window_size+1)
    
    start = max(0, idx - random_window_size)
    end = min(idx + random_window_size + 1, len(words))
    
    target_words = words[start: idx] + words[idx+1: end]
    
    return list(set(target_words))

In [15]:
get_target_words(words_int, 0)

[20171, 54572]

In [16]:
def get_batch(words, batch_size, window_size=5):
    """Construct a generator
    
    Arguments:
        words: words list
        batch_size: batch_size
        window_size: default to 5
    """
    num_batches = len(words) // batch_size
    
    words = words[: num_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch_words = words[idx: idx+batch_size]
        for i in range(0, batch_size):
            targets_y = get_target_words(batch_words, i, window_size)
            # Make sure len(x) is equal to len(y)
            x.extend([batch_words[i]] * len(targets_y))
            y.extend(targets_y)
        yield x, y            

In [ ]:
"""
-------Test-------
"""
batch = get_batch(words_int, 8)
print(type(batch))
i = 0
for x, y in batch:
    print('x = ', x)
    print('y = ', y)
    i += 1
    if i == 2:
        break    

## 6 - Network structure

<img src="images/structure.jpeg" style="width:500;height:300px;">
<caption><center> **Figure 5**: Nueral network structure(We assume the size of the vocabulary is equal to 10000)</center></caption>

In [17]:
len(vocab)

63641

## 6.1 - Input

In [32]:
graph = tf.Graph()
with graph.as_default():
    inputs = tf.placeholder(dtype=tf.int32, shape=[None], name='Inputs')
    labels = tf.placeholder(dtype=tf.int32, shape=[None, 1], name='Labels')

## 6.2 - Embedding

In [33]:
vocab_size = len(vocab)    # size of the vocabulary
embedding_size = 50    # dimentions of word vector

In [34]:
with graph.as_default():
    embedding = tf.Variable(initial_value=tf.random_uniform(shape=[vocab_size, embedding_size], minval=-1, maxval=1), 
                          dtype=tf.float32, name='WordVector')
    embed = tf.nn.embedding_lookup(params=embedding, ids=inputs)    # outpout shape: (None, 50)

## 6.3 - Negative sampling
主要解决computational cost的问题。

方法： tf.nn.sampled_softmax_loss，在softmax层上进行进行采样并计算采样损失。

In [22]:
n_sampled = 20    # 负采样数

In [35]:
with graph.as_default():
    softmax_w = tf.Variable(initial_value=tf.truncated_normal(shape=[vocab_size, embedding_size], stddev=0.1), 
                          dtype=tf.float32, name='u_vector')
    softmax_b = tf.Variable(initial_value=tf.zeros(vocab_size))
    loss = tf.nn.sampled_softmax_loss(weights=softmax_w, biases=softmax_b, inputs=embed, labels=labels, 
                                      num_sampled=n_sampled, num_classes=vocab_size, num_true=1, 
                                      name="sampled_softmax_loss")# intermediate shape: (batch_size, num_true+n_sampled)
    cost = tf.reduce_mean(loss)
    
    global_steps = tf.Variable(initial_value=0, trainable=False)
    train_op = tf.train.AdamOptimizer().minimize(cost, global_step=global_steps)

## 6.4 - Validate

In [ ]:
with graph.as_default():
    # 随机挑选一些单词
    valid_size = 8 
    valid_window = 100
    # 从不同位置各选8个单词
    valid_examples = np.array(random.sample(range(valid_window), valid_size))
    # valid_examples = np.append(valid_examples, random.sample(range(1000,1000+valid_window), valid_size))
        
    valid_examples = [vocab_to_index['mother'], vocab_to_index['cat'], vocab_to_index['china'], vocab_to_index['beijing'], 
                      vocab_to_index['beer'], vocab_to_index['chair'], vocab_to_index['glass'], vocab_to_index['book']]
    
    valid_size = len(valid_examples)
    # 验证单词集
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    
    # normalize embedding
    norm = tf.sqrt(tf.reduce_sum(input_tensor=tf.square(embedding), axis=1, keep_dims=True))
    normalized_embedding = embedding / norm
    # look up the vectors of the valid examples
    valid_embedding = tf.nn.embedding_lookup(params=normalized_embedding, ids=valid_dataset)
    # compute cosine similarity, shape: (valid_size, vocab_size)
    similarity = tf.matmul(a=valid_embedding, b=normalized_embedding, transpose_b=True)

In [26]:
print(valid_examples)

[22691, 578, 21487, 12994, 16446, 8828, 44217, 18015]


In [37]:
with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    saver.restore(sess, './text8_epoch109.ckpt')

    sim = sess.run(similarity)
    k = 10
    top10 = sess.run(tf.nn.top_k(input=sim, k=k))
    
    print(top10.indices.shape)
    for i in range(valid_size):
        valid_word = index_to_vocab[valid_examples[i]]
        log = 'close to %s: ' % valid_word
        for j in range(k):
            log += '%s, ' % index_to_vocab[top10.indices[i][j]]
            # print(top10.indices)
        print(log)

INFO:tensorflow:Restoring parameters from ./text8_epoch109.ckpt
(8, 10)
close to mother: mother, projections, saucers, citation, vinnie, fonts, judean, riefenstahl, ersatz, primes, 
close to cat: cat, faial, allies, finalised, suspense, warship, ipv, finagle, pasha, muslims, 
close to china: china, defamation, ibf, episkopos, diameters, dmc, kune, blue, dunbartonshire, holdings, 
close to beijing: beijing, chooses, alb, evangelicals, dramatization, democrats, mayerling, avoided, radical, griechischen, 
close to beer: beer, constitutionally, mistletoe, versailles, flicker, lycos, socrates, accoutrements, knightsbridge, iname, 
close to chair: chair, vocalised, wil, dribble, durian, vytautas, conjugation, spruance, stagnant, edoardo, 
close to glass: glass, landholdings, chos, telomerase, noticeable, hilton, signifying, discreet, vacancy, olmec, 
close to book: book, sesame, esk, balm, mecca, hogmanay, commodity, sextus, schiller, niles, 


## 6.5 - Train

In [ ]:
epochs = 500
batch_size = 1000
window_size = 10

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    # total_loss = 0
    start_time = time.time()
    print('Training......')
    for epoch in range(epochs):
        batch = get_batch(batch_size=batch_size, window_size=window_size, words=sampled_words)
        for x, y in batch:
            _, loss = sess.run([train_op, cost], feed_dict={inputs: x, labels: np.array(y)[:, None]})
            # total_loss += loss
            
            train_step = sess.run(global_steps)
            if train_step % 100 == 0:
                end_time = time.time()
                print('epoch: {}/{}'.format(epoch, epochs), 
                      'steps: {}'.format(train_step),
                      'loss: {}'.format(loss),
                      'step time: {:.10f} sec/batch'.format((end_time-start_time) / 100))
                start_time = time.time()
            
            if train_step % 1000 == 0:
                sim = sess.run(similarity)
                k = 10
                top10 = sess.run(tf.nn.top_k(input=sim, k=k))
                print(top10.indices.shape)
                for i in range(valid_size):
                    valid_word = index_to_vocab[valid_examples[i]]
                    log = 'close to %s: ' % valid_word
                    for j in range(k):
                        log += '%s, ' % index_to_vocab[top10.indices[i][j]]
                        # print(top10.indices)
                    print(log)
        
        save_path = saver.save(sess, "./text8_epoch{}.ckpt".format(epoch))
        embed_mat = sess.run(normalized_embedding)
    save_path = saver.save(sess, "./checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)